In [6]:
import os
import glob
import pandas as pd
import numpy as np
import os
import pysam
from tqdm import tnrange, tqdm_notebook

In [7]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info/'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/'

In [8]:
all_bams = sorted(glob.glob(os.path.join(input_dir, '*.sorted.bam')))
len(all_bams)

134

In [9]:
def get_readcount(bam_file, mapped=True):
    """
    Parses a bam file idxstats to get the number of reads.
    The BAM file MUST have an index.
    """
    num_reads = pysam.idxstats(
        bam_file
    ).split('\n')
    nums = {}
    for num in num_reads:
        try:
            chrom, chrlen, mapped, unmapped = num.split('\t')
            if mapped:
                nums[chrom] = int(mapped)
            else:
                nums[chrom] = int(mapped) + int(unmapped)
        except ValueError:
            print(num)
    return pd.DataFrame(nums, index=['num']).T.sum().values[0]

In [10]:
progress = tnrange(len(all_bams))
with open(os.path.join(output_dir, 'read_counts.txt'), 'w') as o:
    for bam in all_bams:
        o.write("{}\t{}\t{}\n".format(os.path.basename(bam.replace(".fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.merged.sorted.rmdup.readfiltered.sorted.bam","")), get_readcount(bam, mapped=True), get_readcount(bam, mapped=False)))
        progress.update(1)